In [ ]:
import tensorflow as tf
K = tf.keras.backend
Callback = tf.keras.callbacks.Callback
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau

to_categorical = tf.keras.utils.to_categorical

from sklearn.datasets import make_blobs
from matplotlib import pyplot
from numpy import where

from sklearn.model_selection import train_test_split

import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly

In [ ]:
x, y = make_blobs(n_samples=3000, centers=3, n_features=2, cluster_std=2, random_state=2)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.4, shuffle= True)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
x_train_pd = pd.DataFrame(x_train, columns=['x', 'y'])
y_train_pd = pd.DataFrame(y_train, columns=['class'])

df = pd.concat([x_train_pd, y_train_pd], axis=1)
df["class"] = df["class"].astype(str)

In [ ]:
fig = px.scatter(df, x="x", y="y", color="class")
fig.show()

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [ ]:
def create_model1(learning_rates):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', kernel_initializer='he_uniform'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rates)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

#Train Model และ Plot กราฟ accuracy และ val_accuracy 

In [ ]:
learning_rates = [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]

fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=('learning_rate=1.0', 'learning_rate=0.1', 'learning_rate=0.01', 'learning_rate=0.001', 'learning_rate=0.0001', 'learning_rate=0.00001', 'learning_rate=0.000001', 'learning_rate=0.0000001')
)

for i in range(len(learning_rates)):
    model = create_model1(learning_rates[i])
    
    row = (i//2)+1
    col = (i%2)+1
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=0)
    
    fig.add_trace(go.Scatter(y=history.history['accuracy'], line=dict(color='blue')), row=row, col=col)
    fig.add_trace(go.Scatter(y=history.history['val_accuracy'], line=dict(color='red')), row=row, col=col)
    
    fig.update_xaxes(title_text='Epochs', showgrid=False, row=row, col=col)
    fig.update_yaxes(title_text='Accuracy', showgrid=False, row=row, col=col)
    
fig.update_layout(title_text='Impact of Learning Rate', height=750, showlegend=False)   

# **Momentum**

In [ ]:
def create_model2(momentum):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', kernel_initializer='he_uniform'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=momentum)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
momentums = [0.0, 0.5, 0.9, 0.99]

fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=('Momentum=0.0', 'Momentum=0.5', 'Momentum=0.9', 'Momentum=0.99')
)

for i in range(len(momentums)):
    model = create_model2(momentums[i])
    
    row = (i//2)+1
    col = (i%2)+1
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=0)
    
    fig.add_trace(go.Scatter(y=history.history['accuracy'], line=dict(color='blue')), row=row, col=col)
    fig.add_trace(go.Scatter(y=history.history['val_accuracy'], line=dict(color='red')), row=row, col=col)
    
    fig.update_xaxes(title_text='Epochs', showgrid=False, row=row, col=col)
    fig.update_yaxes(title_text='Accuracy', showgrid=False, row=row, col=col)
    
fig.update_layout(title_text='Impact of Momentum', height=750, showlegend=False)   


# **Learning Rate Decay**

In [ ]:
def decay_lrate(initial_lrate, decay, iteration):
    return initial_lrate * (1.0 / (1.0 + decay * iteration))

In [ ]:
data = []
decays = [0.1, 0.01, 0.001, 0.0001]
learning_rate = 0.01
EPOCH = 200
colors = ['red', 'green', 'blue', 'orange']

for i, decay in enumerate(decays):
    learning_rates = [decay_lrate(learning_rate, decay, i) for i in range(EPOCH)]
    
    h = go.Scatter(y=learning_rates, 
                     mode="lines",
                     line=dict(
                         width=2,
                         color=colors[i]),
                     name=str(decay))
    
    data.append(h)
    
layout1 = go.Layout(title='Learning Rate',
                   xaxis=dict(title='Epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1)

In [ ]:
def create_model3(decay):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', kernel_initializer='he_uniform'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.01, decay=decay)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Decay=0.1', 'Decay=0.01', 'Decay=0.001', 'Decay=0.0001')
)

for i in range(len(decays)):
    model = create_model3(decays[i])
    
    row = (i//2)+1
    col = (i%2)+1
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=0)
    
    fig.add_trace(go.Scatter(y=history.history['accuracy'], line=dict(color='blue')), row=row, col=col)
    fig.add_trace(go.Scatter(y=history.history['val_accuracy'], line=dict(color='red')), row=row, col=col)
    
    fig.update_xaxes(title_text='Epochs', showgrid=False, row=row, col=col)
    fig.update_yaxes(title_text='Accuracy', showgrid=False, row=row, col=col)
    
fig.update_layout(title_text='Impact of Decay', height=750, showlegend=False)   


# **Drop Learning Rate on Plateau**

In [ ]:
class LearningRateMonitor(Callback):
    def on_train_begin(self, logs={}):
        self.learning_rates = list()
 
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        learning_rate = float(K.get_value(self.model.optimizer.lr))
        self.learning_rates.append(learning_rate)

In [ ]:
def create_model4(patience):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', kernel_initializer='he_uniform'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.01, decay=decay)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience)
    lrm = LearningRateMonitor()
    

    return model, rlrp, lrm

In [ ]:
patiences = [2, 5, 10, 15]

learning_rate_list=[]
accuracy_list=[]
loss_list=[]

for i in range(len(patiences)):
    model, rlrp, lrm = create_model4(patiences[i])
    
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=0, callbacks=[rlrp, lrm])
    lrm.learning_rates
    
    learning_rate_list.append(lrm.learning_rates)
    accuracy_list.append(history.history['val_accuracy'])
    loss_list.append(history.history['val_loss'])

In [ ]:
def patiences_plot(y, title_text):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Patience=2', 'Patience=5', 'Patience=10', 'Patience=15')
    )

    for i in range(len(patiences)):
        model = create_model4(patiences[i])

        row = (i//2)+1
        col = (i%2)+1
        fig.add_trace(go.Scatter(y=y[i], line=dict(color='red')), row=row, col=col)
        fig.update_xaxes(title_text='Epochs', showgrid=False, row=row, col=col)
        fig.update_yaxes(title_text=title_text, showgrid=False, row=row, col=col)

    fig.update_layout(title_text='Impact of Patience', height=750, showlegend=False)
    fig.show()

In [ ]:
patiences_plot(learning_rate_list, 'Learning Rate')

In [ ]:
patiences_plot(loss_list, 'val_loss')

In [ ]:
patiences_plot(accuracy_list, 'val_accuracy')

# **Adaptive Learning Rates Gradient Descent**

In [ ]:
def create_model5(optimizer):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', kernel_initializer='he_uniform'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
optimizers = ['sgd', 'rmsprop', 'adagrad', 'adam']

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Optimizer=sgd', 'Optimizer=rmsprop', 'Optimizer=adagrad', 'Optimizer=adam')
)

for i in range(len(optimizers)):
    model = create_model5(optimizers[i])
    
    row = (i//2)+1
    col = (i%2)+1
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, verbose=0)
    
    fig.add_trace(go.Scatter(y=history.history['accuracy'], line=dict(color='blue')), row=row, col=col)
    fig.add_trace(go.Scatter(y=history.history['val_accuracy'], line=dict(color='red')), row=row, col=col)
    
    fig.update_xaxes(title_text='Epochs', showgrid=False, row=row, col=col)
    fig.update_yaxes(title_text='Accuracy', showgrid=False, row=row, col=col)
    
fig.update_layout(title_text='Impact of Optimizer', height=750, showlegend=False)   
